In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2yo91gft
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2yo91gft
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10733 sha256=25d7336482e00933626da974c4579936e1bcd2169b147b54a7d099795d56a4e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-3gk1mtvt/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpbg0qy346".


In [2]:
!apt-get update
!apt-get install -y nvidia-cuda-toolkit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,110 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,616 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,482 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.

In [3]:
%%writefile matrix_multiplication.cu
#include <stdlib.h>
#include <stdio.h>
#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <time.h>
#include <math.h>

// Define matrix indexing for column-major order
#define index(i,j,ld) (((j)*(ld))+(i))

// Initialize matrices with smaller values for numerical stability
void initializeMatrix(float *matrix, int size) {
    for (int i = 0; i < size; i++) {
        for (int j = 0; j < size; j++) {
            matrix[index(i, j, size)] = (float)(i + j) / size;
        }
    }
}

// CPU matrix multiplication (column-major order)
void cpuMatrixMultiplication(float *A, float *B, float *C, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            C[index(i, j, n)] = 0.0f;
            for (int k = 0; k < n; k++) {
                C[index(i, j, n)] += A[index(i, k, n)] * B[index(k, j, n)];
            }
        }
    }
}

int main() {
    int sizes[] = {256, 512, 1024};
    int numSizes = 3;

    for (int s = 0; s < numSizes; s++) {
        int size = sizes[s];
        printf("\nRunning matrix multiplication for size: %d x %d\n", size, size);

        // Allocate host memory (aligned to 32-byte boundaries)
        float *A = (float*)aligned_alloc(32, size * size * sizeof(float));
        float *B = (float*)aligned_alloc(32, size * size * sizeof(float));
        float *C_cpu = (float*)aligned_alloc(32, size * size * sizeof(float));
        float *C_gpu = (float*)aligned_alloc(32, size * size * sizeof(float));

        // Initialize matrices A and B
        initializeMatrix(A, size);
        initializeMatrix(B, size);

        // Timing CPU matrix multiplication
        clock_t start_cpu = clock();
        cpuMatrixMultiplication(A, B, C_cpu, size);
        clock_t end_cpu = clock();
        double time_cpu = ((double)(end_cpu - start_cpu)) / CLOCKS_PER_SEC;
        printf("CPU Matrix Multiplication Time: %f seconds\n", time_cpu);

        // Allocate device memory
        float *d_A, *d_B, *d_C;
        cudaMalloc((void**)&d_A, size * size * sizeof(float));
        cudaMalloc((void**)&d_B, size * size * sizeof(float));
        cudaMalloc((void**)&d_C, size * size * sizeof(float));

        // Copy matrices from host to device
        cudaMemcpy(d_A, A, size * size * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_B, B, size * size * sizeof(float), cudaMemcpyHostToDevice);

        // Create cuBLAS handle
        cublasHandle_t handle;
        cublasCreate(&handle);

        float alpha = 1.0f;
        float beta = 0.0f;

        // Timing GPU matrix multiplication using cuBLAS
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);

        // Matrix multiplication using cuBLAS (column-major order)
        cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, size, size, size, &alpha, d_B, size, d_A, size, &beta, d_C, size);

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float time_gpu;
        cudaEventElapsedTime(&time_gpu, start, stop);
        printf("GPU Matrix Multiplication Time (cuBLAS): %f milliseconds\n", time_gpu);

        // Copy result back to host
        cudaMemcpy(C_gpu, d_C, size * size * sizeof(float), cudaMemcpyDeviceToHost);

        // Verify the results using relative error
        int errors = 0;
        float max_relative_error = 1e-4;
        for (int i = 0; i < size * size; i++) {
            float relative_error = fabs(C_cpu[i] - C_gpu[i]) / fmax(fabs(C_cpu[i]), fabs(C_gpu[i]));
            if (relative_error > max_relative_error) {
                errors++;
            }
        }
        if (errors == 0) {
            printf("Results verified successfully for size %d x %d\n", size, size);
        } else {
            printf("Discrepancies found in the results for size %d x %d\n", size, size);
        }

        // Clean up
        cublasDestroy(handle);
        cudaFree(d_A);
        cudaFree(d_B);
        cudaFree(d_C);
        free(A);
        free(B);
        free(C_cpu);
        free(C_gpu);
    }

    return 0;
}

Writing matrix_multiplication.cu


In [4]:
# Compile the CUDA code
!nvcc matrix_multiplication.cu -lcublas -o matrix_multiplication

In [5]:
# Run the compiled executable
!./matrix_multiplication


Running matrix multiplication for size: 256 x 256
CPU Matrix Multiplication Time: 0.078256 seconds
GPU Matrix Multiplication Time (cuBLAS): 95.807678 milliseconds
Results verified successfully for size 256 x 256

Running matrix multiplication for size: 512 x 512
CPU Matrix Multiplication Time: 0.983451 seconds
GPU Matrix Multiplication Time (cuBLAS): 0.280416 milliseconds
Results verified successfully for size 512 x 512

Running matrix multiplication for size: 1024 x 1024
CPU Matrix Multiplication Time: 11.857524 seconds
GPU Matrix Multiplication Time (cuBLAS): 0.984160 milliseconds
Results verified successfully for size 1024 x 1024
